In [1]:
import cobra
import pandas as pd
import escher

# Dolosigranulum pigrum (DP)
# Staphylococcus aureus (SA)

In [2]:
# Import SNM3 compund files
snm3 = pd.read_csv("SNM3.csv", sep=";")
snm3

,Compound,BiGG,ModelSeed,KEGG
0,Alanine,ala__L,cpd00035,C00041
1,Arginine,arg__L,cpd00051,C00062
2,Cysteine,cys__L,cpd00084,C00097
3,Glutamic acid,glu__L,cpd00023,C00025
4,Glycine,gly,cpd00033,C00037
5,Histidine,his__L,cpd00119,C00135
6,Leucine,leu__L,cpd00107,C00123
7,Lysine,lys__L,cpd00039,C00047
8,Ornithine,orn,cpd00064,C00077
9,Phenylalanine,phe__L,cpd00066,C00079


## 2) SNM growth testing

In [3]:
model_DP = cobra.io.read_sbml_model("models/consistent_DP.xml")
model_SA = cobra.io.read_sbml_model("models/consistent_iYS854.xml")

We will use as default:
* Lower bound -10
* Lower bound O2 -20
* Lower bound Fe -0.1
* Upper bound: 1000

In [4]:
print(f"Number of exchange reactions: {len(model_DP.exchanges)}")
print(f"Number of exchange reactions: {len(model_SA.exchanges)}")


Number of exchange reactions: 64
Number of exchange reactions: 83


In [5]:
# Checking bounds
for f in model_DP.exchanges:
    if f.id == "EX_o2_e":
        f.lower_bound=-20
    elif f.id == "EX_fe2_e":
        f.lower_bound=-0.1
    else: 
        f.lower_bound=-10
    f.upper_bound=1000

In [6]:
# Checking bounds
for f in model_SA.exchanges:
    if f.id == "EX_o2_e":
        f.lower_bound=-20
    elif f.id == "EX_fe2_e":
        f.lower_bound=-0.1
    else: 
        f.lower_bound=-10
    f.upper_bound=1000

In [7]:
# Check for growth with all exchanges turend on
sol = model_DP.optimize()
print(sol.objective_value)

0.38177836171798285


In [8]:
# Check for growth with all exchanges turend on
sol = model_SA.optimize()
print(sol.objective_value)

5.026388539834129


From this summary we observe that for maximal growth the bacterium needs most importantly glucose, L-glutamate, phosphate, glycin, L-arginine, oxygen and L-leucin. As checked by the next for loop all the necessary metabolites are available in SNM3, except L-Isoleucine and Meso-2,6-Diaminoheptanedioate.

One further have to check if those two metabolites are essential for growth or not.

So they are not synthesizable. We will now test if the model can grow on SNM3 media, which we define as following:
* The SNM3 media contains all metbolites listed in "snm3.csv" that **have a BiGG ID** listed and **are exchange reactions of the model**.
* The concentration is set to 10, except for o2 it is set to 20. This is equivalent with having all inward exchange reactions that are in the SNM3 media to -10 (o2 to -20) else to 0.

In [9]:
# Create full snm3 media 
snm3_media = dict()
ex_ids = [e.id for e in model_DP.exchanges]
for id in snm3["BiGG"]:
    id = "EX_" + str(id) + "_e"
    if id in ex_ids:
        snm3_media[id] = 10
        if id == "EX_o2_e":
            snm3_media[id] = 20
        if "EX_fe" in id:
            snm3_media[id] = 0.1
# Tets for growth...
with model_DP:
    model_DP.medium = snm3_media
    print(model_DP.slim_optimize())

0.0


In [10]:
# This model need these metabolites for growth, hence we will include them in the medium!
snm3_media["EX_ile__L_e"] = 10
snm3_media["EX_met__L_e"] = 10
snm3_media["EX_26dap__M_e"] = 10

with model_DP:
    model_DP.medium = snm3_media
    print(model_DP.slim_optimize())
    summary = model_DP.summary()
    
summary

0.28236539253224846


Metabolite,Reaction,Flux,C-Number,C-Flux
26dap__M_e,EX_26dap__M_e,0.02734,7,0.27%
4abz_e,EX_4abz_e,0.0001829,7,0.00%
ala__L_e,EX_ala__L_e,0.6182,3,2.63%
arg__L_e,EX_arg__L_e,0.4395,6,3.74%
ca2_e,EX_ca2_e,0.01468,0,0.00%
cl_e,EX_cl_e,0.01659,0,0.00%
cobalt2_e,EX_cobalt2_e,0.009982,0,0.00%
cu2_e,EX_cu2_e,0.009257,0,0.00%
cys__L_e,EX_cys__L_e,0.02654,3,0.11%
fe2_e,EX_fe2_e,0.02107,0,0.00%


In [16]:
from cobra.flux_analysis import flux_variability_analysis
fva_DP = cobra.flux_analysis.flux_variability_analysis(model_DP)

In [ ]:
# Sticking them together
df_fluxes = fva.copy()
df_fluxes["flux"] = sol.fluxes
df_fluxes.to_excel("fluxes_DP.xlsx")
df_fluxes.head()

In [17]:
# Exchange fluxes fva, for all non-zero exchanges
fva_ex_DP = model_DP.summary(fva=1.0)
fva_ex_DP = fva_ex_DP.to_frame()

In [ ]:
# Get necessary values for plotting
x = []
y_fluxes = []
y_min_flux = []
y_max_flux = []
for _, data in fva_ex.iterrows():
    id = str(data.metabolite[:-2])
    f = data.factor*data.flux 
    f_min = data.factor*data.minimum 
    f_max = data.factor*data.maximum

    if id in snm3["BiGG"].tolist() and not (f_min == 0 and f_max == 0):
        x.append(id + "_e")
        y_fluxes.append(f)
        y_min_flux.append(f_min)
        y_max_flux.append(f_max)



In [11]:
# Create full snm3 media 
snm3_media = dict()
ex_ids = [e.id for e in model_SA.exchanges]
for id in snm3["BiGG"]:
    id = "EX_" + str(id) + "_e"
    if id in ex_ids:
        snm3_media[id] = 10
        if id == "EX_o2_e":
            snm3_media[id] = 20
        if "EX_fe" in id:
            snm3_media[id] = 0.1
        if "EX_orn" in id:
            snm3_media[id] = 0.0   
            
# Test for growth...
with model_SA:
    model_SA.medium = snm3_media
    print(model_SA.slim_optimize())

2.5586946126134027


In [12]:
with model_SA:
    model_SA.medium = snm3_media
    print(model_SA.slim_optimize())
    summary = model_SA.summary()
    
summary

2.5586946126134027


Metabolite,Reaction,Flux,C-Number,C-Flux
arg__L_e,EX_arg__L_e,4.354,6,12.06%
cl_e,EX_cl_e,0.01267,0,0.00%
cys__L_e,EX_cys__L_e,0.4614,3,0.64%
fe2_e,EX_fe2_e,0.03776,0,0.00%
glc__D_e,EX_glc__D_e,10,6,27.70%
glu__L_e,EX_glu__L_e,10,5,23.09%
his__L_e,EX_his__L_e,0.1542,6,0.43%
k_e,EX_k_e,0.4751,0,0.00%
leu__L_e,EX_leu__L_e,0.8035,6,2.23%
lys__L_e,EX_lys__L_e,0.9223,6,2.56%


In [15]:
from cobra.flux_analysis import flux_variability_analysis
flux_variability_analysis(model_SA, model_SA.reactions)

,minimum,maximum
10M3HDHL,0.015238,0.015238
10M3OACPO,0.290758,0.290758
10M3OUO,0.015238,0.015238
10MDOD,0.290758,0.290758
10MTDAO,0.290758,0.290758
...,...,...
GLYCLTt,-0.002242,-0.002242
r2535,-178.841332,10.000000
UAG2EMA,0.000000,172.348617
GLUt3,0.000000,1180.388228
